In [12]:
import pandas as pd
import numpy as np
import collections
from datetime import datetime

## Среднее время на кластер

In [13]:
df = pd.read_csv("Team 1.csv")
df = df.rename(columns={"case concept:name": "case:concept:name",
                   "event concept:name": "concept:name",
                   "event time:timestamp": "time:timestamp"});

#### Word2Vec

In [14]:
import gensim
from tqdm.auto import tqdm

In [15]:
word_two_vec_df = df.groupby("case:concept:name")["concept:name"].apply(' '.join).reset_index()
word_two_vec_df["concept:name"] = word_two_vec_df["concept:name"].str.split(' ')

model = gensim.models.Word2Vec(word_two_vec_df["concept:name"], min_count=1, size=100, seed=42)

features = [np.mean([model.wv[word] for word in sent], axis=0).tolist() for sent in word_two_vec_df["concept:name"]] 

#### DBSCAN

In [16]:
from sklearn.cluster import DBSCAN

In [17]:
eps = 0.1
min_samples = 2


clustering = DBSCAN(eps = eps, min_samples=min_samples).fit(features)
labels = clustering.labels_

#### Подготовка DataFrame

In [18]:
df = df[["case:concept:name","concept:name","time:timestamp"]]
df["newColumn"] = df["concept:name"] +"@"+ df["time:timestamp"].apply(str)

In [19]:
proc = df.groupby('case:concept:name')["newColumn"].apply(list)
proc = proc.apply(lambda x: sorted([e.split("@") for e in x], key=lambda x: datetime.strptime(x[1], '%d-%m-%Y %H:%M:%S.%f')))

In [20]:
rows_list = []
for eventId in proc.index:
    event = proc[eventId]
    rows_list.append([eventId, event[0][1], event[-1][1]])
    
df_time = pd.DataFrame(rows_list, columns=['id', 'time start', 'time end'])

In [21]:
df_time['time start'] = df_time['time start'].apply(lambda x: pd.Timestamp(x))
df_time['time end'] = df_time['time end'].apply(lambda x: pd.Timestamp(x))
df_time["timedelta"] = pd.to_timedelta((df_time['time end'] - df_time['time start'].min(axis=0))).dt.total_seconds()
df_time['labels'] = labels

In [22]:
df_time.head(5)

,id,time start,time end,timedelta,labels
0,2000000005_00001,2018-08-01 10:08:00,2018-02-08 09:19:00,2.210146e+09,0
1,2000000005_00002,2018-08-01 10:08:00,2018-08-01 11:09:00,2.225186e+09,1
2,2000000009_00001,2018-02-01 23:59:00,2018-03-15 14:13:00,2.213187e+09,2
3,2000000011_00001,2018-09-01 16:54:00,2019-01-17 11:59:00,2.239790e+09,3
4,2000000019_00001,2018-12-01 09:52:00,2019-01-17 11:59:00,2.239790e+09,-1


#### Группируем по класстерам и считаем среднее время

In [23]:
df_time_groupby = df_time.groupby(['labels']).mean()["timedelta"]

In [24]:
df_time_groupby

labels
-1      2.232684e+09
 0      2.221292e+09
 1      2.229679e+09
 2      2.229260e+09
 3      2.239790e+09
            ...     
 677    2.241521e+09
 678    2.249553e+09
 679    2.237708e+09
 680    2.233569e+09
 681    2.250823e+09
Name: timedelta, Length: 683, dtype: float64

In [25]:
df_time_groupby.describe()

count    6.830000e+02
mean     2.230739e+09
std      8.947753e+06
min      2.207234e+09
25%      2.224178e+09
50%      2.229311e+09
75%      2.237013e+09
max      2.262009e+09
Name: timedelta, dtype: float64

## Самый частый цикл

In [26]:
proc = df.groupby('case:concept:name')["concept:name"].apply(list)

In [27]:
c = collections.Counter()

for p in proc:
    for i in range(len(p)):
        for j in range(i + 1, len(p)):
            if p[i] == p[j]:
                c[tuple(p[i:j])] += 1
                break

In [28]:
c.most_common(10)

[(('Record Service Entry Sheet',), 24116),
 (('Record Goods Receipt',), 8894),
 (('Record Service Entry Sheet', 'Record Goods Receipt'), 3364),
 (('Record Goods Receipt', 'Record Service Entry Sheet'), 1875),
 (('Record Invoice Receipt',), 1580),
 (('Record Service Entry Sheet',
   'Record Goods Receipt',
   'Record Goods Receipt'),
  1108),
 (('Record Goods Receipt',
   'Record Service Entry Sheet',
   'Record Service Entry Sheet'),
  1059),
 (('Clear Invoice',), 645),
 (('Record Goods Receipt',
   'Record Service Entry Sheet',
   'Record Service Entry Sheet',
   'Record Service Entry Sheet'),
  544),
 (('Change Quantity',), 488)]

## Feature Importance

In [29]:
from catboost import CatBoostRegressor

In [30]:
df = pd.read_csv("Team 1.csv")

#### Получем таблицу процессов без различающихся внутри процесса столбцов

In [31]:
train_data = df.drop_duplicates('case concept:name').drop(columns = ['event User','event org:resource', 'event concept:name', 'event Cumulative net worth (EUR)', 'event time:timestamp', 'eventID'])

#### Подготовка данных

In [32]:
cat_features = list(map(lambda x: x[0], list(filter(lambda x: x[1] == True, enumerate((train_data.dtypes == object))))))

In [33]:
train_labels = train_data.merge(df_time, left_on='case concept:name', right_on='id')["timedelta"].values

In [34]:
%%capture
model = CatBoostRegressor(iterations=200, learning_rate=1, depth=5)
model.fit(train_data.replace(np.nan, '', regex=True).values, train_labels, cat_features)

#### Выводим важность фичей

In [35]:
sorted(list(zip(train_data.columns, model.get_feature_importance())), key = lambda x: -x[1])

[('case Purchasing Document', 61.33385606411267),
 ('case Vendor', 13.109925306272283),
 ('case Sub spend area text', 6.072288999314436),
 ('case Name', 5.940326630072922),
 ('case Spend classification text', 3.8864969853157354),
 ('case Spend area text', 2.504314503124203),
 ('case Item Type', 2.439540597249637),
 ('case Item', 1.7384141176061412),
 ('case Item Category', 1.636948926625121),
 ('case Document Type', 0.884426942469063),
 ('case Company', 0.3424064237471488),
 ('case GR-Based Inv. Verif.', 0.10797118229694369),
 ('case concept:name', 0.0030833217936988232),
 ('case Purch. Doc. Category name', 0.0),
 ('case Source', 0.0),
 ('case Goods Receipt', 0.0)]

##### Добавляем фичи

In [36]:
for col in ['event User','event org:resource', 'event concept:name', 'event Cumulative net worth (EUR)']:
    dict_val = df.groupby('case concept:name')[col].apply(list)
    val = []
    
    for case_concept_name in df['case concept:name']:
        val.append(dict_val[case_concept_name])
    
    train_data[col] = pd.DataFrame({col: val})

#### Проверяем стобцы на равенство

In [38]:
set(train_data["event User"].values == train_data['event org:resource'].values)

{True}

##### Уникальные сущности по ролям

In [39]:
def unic_worker(s):
    return s[:s.find("_")]

In [40]:
train_data["len event User"] = train_data["event User"].apply(lambda x: len(x))
train_data["len set event User"] = train_data["event User"].apply(lambda x: len(set(x)))
train_data["len set unic_worker event User"] = train_data["event User"].apply(lambda x: len(set(unic_worker(str(x)))))
train_data["len set event concept:name"] = train_data["event concept:name"].apply(lambda x: len(set(x)))
train_data["len set event Cumulative net worth (EUR)"] = train_data["event Cumulative net worth (EUR)"].apply(lambda x: len(set(x)))

In [41]:
train_data = train_data.drop(columns = ['event User','event org:resource', 'event concept:name', 'event Cumulative net worth (EUR)'])

In [42]:
train_labels = train_data.merge(df_time, left_on='case concept:name', right_on='id')["timedelta"].values

In [43]:
%%capture
model = CatBoostRegressor(iterations=200, learning_rate=1, depth=5)
model.fit(train_data.replace(np.nan, '', regex=True).values, train_labels, cat_features)

#### Выводим важность фичей

In [44]:
sorted(list(zip(train_data.columns, model.get_feature_importance())), key = lambda x: -x[1])

[('case Purchasing Document', 55.46998774004838),
 ('case Vendor', 9.733886718315272),
 ('len set event concept:name', 6.403020734284109),
 ('case Name', 6.1437787567966256),
 ('len event User', 5.761434097296831),
 ('len set event User', 5.560438600844259),
 ('case Sub spend area text', 2.7323260521932853),
 ('case Item', 1.4000158156160694),
 ('case Spend classification text', 1.3788963811505632),
 ('case Spend area text', 1.1559273018026817),
 ('case Item Type', 1.132780946026765),
 ('case Item Category', 1.022657509735084),
 ('case Document Type', 0.7208759594603795),
 ('len set unic_worker event User', 0.7182457473404563),
 ('len set event Cumulative net worth (EUR)', 0.4383656096323184),
 ('case GR-Based Inv. Verif.', 0.18086572947717272),
 ('case Company', 0.043889656460851544),
 ('case concept:name', 0.002606643518898259),
 ('case Purch. Doc. Category name', 0.0),
 ('case Source', 0.0),
 ('case Goods Receipt', 0.0)]